<a href="https://colab.research.google.com/github/Anastasiyka116/IT-school-Case-Task/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/Data_2.1.csv')
df

,user_id,install_date,subscription_name,payment_type,event_date,country,device_family,device_type,os_version,language,system_theme,refund
0,ed7bba31-2aa0-411d-ab51-727e91bb91af,2025-06-01,subs_weekly_7day_free_trial_4_99,free_trial,2025-06-01,Argentina,Apple iPhone,Apple iPhone 12 Pro,18.7.5,Spanish,light,NaN
1,b28a53ac-a75b-401e-a839-56b68fc84581,2025-06-01,subs_weekly_7day_free_trial_4_99,free_trial,2025-06-01,United States,Apple iPhone,Apple iPhone 15 Pro Max,18.7.5,English,light,NaN
2,2d4e82cc-845e-408d-a2c8-545648349c4f,2025-06-01,subs_weekly_7day_free_trial_4_99,free_trial,2025-06-01,Brazil,Apple iPhone,Apple iPhone 11 Pro Max,18.7.5,Portuguese,dark,NaN
3,e9d0bd86-138e-4e78-ac99-e45b85518567,2025-06-01,subs_weekly_7day_free_trial_4_99,free_trial,2025-06-01,Japan,Apple iPhone,Apple iPhone 13 mini,18.7.5,Japanese,dark,NaN
4,e6c5188d-3152-423a-9c9e-9846fb81f420,2025-06-01,subs_weekly_7day_free_trial_4_99,free_trial,2025-06-01,Australia,Apple iPhone,Apple iPhone 16 Pro,18.7.5,English,light,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7271,6df91c08-cdbf-4fbb-926f-047d4c738fe4,2025-06-04,subs_weekly_7day_free_trial_4_99,recurrent,2025-11-04,Panama,Apple iPhone,Apple iPhone 13 Pro Max,18.7.5,Spanish,dark,NaN
7272,84fe9b35-279c-4934-b44c-1e7449795d71,2025-06-03,subs_weekly_7day_free_trial_4_99,recurrent,2025-11-08,United States,Apple iPhone,Apple iPhone 15 Pro Max,17.7.2,English,dark,NaN
7273,6df91c08-cdbf-4fbb-926f-047d4c738fe4,2025-06-04,subs_weekly_7day_free_trial_4_99,recurrent,2025-11-10,Panama,Apple iPhone,Apple iPhone 13 Pro Max,18.7.5,Spanish,dark,NaN
7274,84fe9b35-279c-4934-b44c-1e7449795d71,2025-06-03,subs_weekly_7day_free_trial_4_99,recurrent,2025-11-15,United States,Apple iPhone,Apple iPhone 15 Pro Max,17.7.2,English,light,NaN


In [12]:
tier1 = ['United States', 'Canada', 'United Kingdom', 'Australia']
rw = ['Sweden', 'Ireland', 'Belgium', 'France', 'Italy', 'Austria', 'Germany', 'Denmark', 'Norway', 'Switzerland', 'Netherlands', 'New Zealand', 'Finland', 'Luxembourg', 'Iceland']
spanish = ['Chile', 'Venezuela', 'Spain', 'Puerto Rico', 'Bolivia', 'Dominican Republic', 'Ecuador', 'Panama', 'Paraguay', 'Uruguay', 'Argentina']

def get_region(country):
    if country in tier1:
        return 'Tier1'
    elif country in rw:
        return 'RW'
    elif country in spanish:
        return 'Spanish'
    else:
        return 'Other'
df['region'] = df['country'].apply(get_region)


# Question 1: How does the behavior differ among geographies for our subscription? Calculate the expected LTV in each region

In [13]:
# Спочатку створимо зручні колонки-флаги (1 або 0)
df['is_trial'] = (df['payment_type'] == 'free_trial').astype(int)
df['is_recurrent'] = (df['payment_type'] == 'recurrent').astype(int)

user_stats = df.groupby(['user_id', 'region']).agg(
    trials=('is_trial', 'max'),             # 1 якщо користувач брав тріал, 0 якщо ні
    recurrent_payments=('is_recurrent', 'sum') # Загальна кількість оплат користувача
).reset_index()

# Обмежуємо кількість оплат до 5 (LTV рахуємо за 6 тижнів: 1 тріал + 5 оплат)
user_stats['capped_recurrent'] = user_stats['recurrent_payments'].clip(upper=5)

region_stats = user_stats.groupby('region').agg(
    total_trials=('trials', 'sum'),
    total_recurrent=('capped_recurrent', 'sum')
).reset_index()

dev_proceeds = 4.99 * 0.70

# Розрахунок LTV: (Всі оплати * 3.493) / Кількість тріалів
region_stats['expected_ltv'] = round(
    (region_stats['total_recurrent'] * dev_proceeds) / region_stats['total_trials'],
    2
)
region_stats = region_stats.sort_values(by='expected_ltv', ascending=False).reset_index(drop=True)
print(region_stats)

    region  total_trials  total_recurrent  expected_ltv
0    Tier1           852             1274          5.22
1       RW           229              323          4.93
2  Spanish           167              196          4.10
3    Other          2093             2142          3.57


# Question 2: What other important features of different groups of users (by language, device, iOS, etc.) have you noticed? Can you give reasons why this happens?

System Theme (Light vs. Dark Mode):

In [52]:
def calculate_conversion_by_group(dataframe, group_column, min_trials=30):
    # Рахуємо кількість унікальних користувачів, які взяли тріал у кожній групі
    trials = dataframe[dataframe['payment_type'] == 'free_trial'].groupby(group_column)['user_id'].nunique().reset_index()
    trials.columns = [group_column, 'total_trials']

    # Знаходимо унікальних користувачів, які зробили хоча б 1 оплату (recurrent)
    payers_list = dataframe[dataframe['payment_type'] == 'recurrent']['user_id'].unique()

    # Відфільтровуємо лише тих користувачів, які взяли тріал і потім заплатили
    df_payers = dataframe[(dataframe['payment_type'] == 'free_trial') & (dataframe['user_id'].isin(payers_list))]

    # Рахуємо їхню кількість по групах
    payers = df_payers.groupby(group_column)['user_id'].nunique().reset_index()
    payers.columns = [group_column, 'successful_payers']

    # Об'єднуємо таблицю тріалів і таблицю оплат
    stats = pd.merge(trials, payers, on=group_column, how='left')
    stats['successful_payers'] = stats['successful_payers'].fillna(0)

    # Фільтруємо групи, де мало даних (менше 30 тріалів), щоб уникнути шуму
    stats = stats[stats['total_trials'] >= min_trials].copy()

    # Рахуємо відсоток конверсії (CTFP)
    stats['CTFP (%)'] = round((stats['successful_payers'] / stats['total_trials']) * 100, 2)

    # Сортуємо від найвищої конверсії до найнижчої
    stats = stats.sort_values(by='CTFP (%)', ascending=False).reset_index(drop=True)
    return stats
print("--- Конверсія за кольором системи пристрою (system_theme) ---")
device_stats = calculate_conversion_by_group(df, 'system_theme')
print("Конверсія в залежності від теми:")
print(device_stats)

--- Конверсія за кольором системи пристрою (system_theme) ---
Конверсія в залежності від теми:
  system_theme  total_trials  successful_payers  CTFP (%)
0        light          1981                750     37.86
1         dark          1274                398     31.24


Language Performance:

In [53]:
print("--- Конверсія за мовою системи пристрою (Language) ---")
device_stats = calculate_conversion_by_group(df, 'language')
print("Конверсія в залежності від мови:")
print(device_stats)

--- Конверсія за мовою системи пристрою (Language) ---
Конверсія в залежності від мови:
      language  total_trials  successful_payers  CTFP (%)
0       Hebrew            30               13.0     43.33
1      Italian            57               23.0     40.35
2      Spanish           425              170.0     40.00
3       French            84               32.0     38.10
4      English          1313              488.0     37.17
5   Portuguese           461              154.0     33.41
6         Thai           142               47.0     33.10
7       German            47               14.0     29.79
8       Arabic           267               79.0     29.59
9      Turkish           112               28.0     25.00
10    Japanese           215               50.0     23.26


Вплив моделі пристрою (Device Type)

In [54]:
print("--- Конверсія за моделлю пристрою (Device Type) ---")
device_stats = calculate_conversion_by_group(df, 'device_type')
print("ТОП-5 найкращих пристроїв:")
print(device_stats.head(5))
print("\nТОП-5 найгірших пристроїв:")
print(device_stats.tail(5))

--- Конверсія за моделлю пристрою (Device Type) ---
ТОП-5 найкращих пристроїв:
               device_type  total_trials  successful_payers  CTFP (%)
0      Apple iPhone 16 Pro           144               64.0     44.44
1  Apple iPhone 15 Pro Max           306              132.0     43.14
2      Apple iPhone 14 Pro           112               48.0     42.86
3  Apple iPhone 16 Pro Max           364              153.0     42.03
4          Apple iPhone 16           109               45.0     41.28

ТОП-5 найгірших пристроїв:
                device_type  total_trials  successful_payers  CTFP (%)
17  Apple iPhone 12 Pro Max           147               39.0     26.53
18      Apple iPhone 12 Pro            61               16.0     26.23
19     Apple iPhone 13 mini            38                7.0     18.42
20  Apple iPhone 11 Pro Max            60               11.0     18.33
21      Apple iPhone 11 Pro            79               14.0     17.72


Вплив версії ОС (OS Version)

In [55]:
print("\n--- Конверсія за версією операційної системи (OS Version) ---")
os_stats = calculate_conversion_by_group(df, 'os_version')
print(os_stats)


--- Конверсія за версією операційної системи (OS Version) ---
  os_version  total_trials  successful_payers  CTFP (%)
0     18.7.5          2963             1070.0     36.11
1    16.7.14            56               18.0     32.14
2     17.7.2           321               82.0     25.55


Refund Rate

In [28]:
df['refund'].value_counts()

,count
refund,
yes,346


In [40]:
user_stats = df.groupby(['user_id', 'region']).agg(
    refund=('refund', 'count'),
    trials=('is_trial', 'max')
).reset_index()
region_refund_stats = user_stats.groupby('region').agg(
    total_trials=('trials', 'sum'),
    total_refund=('refund', 'sum')
).reset_index()
region_refund_stats['refund_rate'] = round(region_refund_stats['total_refund']*100/region_refund_stats['total_trials'],2)
region_refund_stats = region_refund_stats.sort_values(by='refund_rate', ascending=False).reset_index(drop=True)
region_refund_stats

,region,total_trials,total_refund,refund_rate
0,Tier1,852,121,14.20
1,RW,229,26,11.35
2,Spanish,167,16,9.58
3,Other,2093,183,8.74


Question 3: Estimate how much we would be able to earn in 6 and 12 months for this cohort of users. Use whatever growth assumptions you find reasonable but explain your logic

Chain Ratio Forecasting (Cohort Analysis)

In [43]:
#Відомі дані з нашої воронки (кількість користувачів, що зробили 1, 2, 3, 4, 5 оплат)
users_w1 = 1170
users_w2 = 870
users_w3 = 724
users_w4 = 620
users_w5 = 551
dev_proceeds = 4.99 * 0.70

# 1. Розраховуємо історичні ланцюгові коефіцієнти (Chain Ratios)
r1_to_2 = users_w2 / users_w1  # ~0.7436
r2_to_3 = users_w3 / users_w2  # ~0.8322
r3_to_4 = users_w4 / users_w3  # ~0.8564
r4_to_5 = users_w5 / users_w4  # ~0.8887

print("--- Історичні ланцюгові коефіцієнти ---")
print(f"Тиждень 1 -> 2: {r1_to_2:.4f}")
print(f"Тиждень 2 -> 3: {r2_to_3:.4f}")
print(f"Тиждень 3 -> 4: {r3_to_4:.4f}")
print(f"Тиждень 4 -> 5: {r4_to_5:.4f}\n")
projected_users = [users_w1, users_w2, users_w3, users_w4, users_w5]
current_ratio = r4_to_5
# 2. Прогнозуємо з 6-го по 52-й тиждень (1 рік)
for week in range(6, 53):
    # Якщо коефіцієнт менший за 97% (0.97), ми потроху його збільшуємо.
    # Це імітує "вирівнювання кривої відтоку" (найлояльніші користувачі залишаються).
    if current_ratio < 0.97:
        # З кожним тижнем коефіцієнт зростає на 10% від відстані, що залишилась до 0.97
        current_ratio = current_ratio + (0.97 - current_ratio) * 0.10
    next_week_users = projected_users[-1] * current_ratio
    projected_users.append(next_week_users)
earned_6_months = sum(projected_users[:26]) * dev_proceeds
earned_12_months = sum(projected_users[:52]) * dev_proceeds
print("--- Прогноз ---")
print(f"Користувачів на 26-й тиждень: {int(projected_users[25])}")
print(f"Користувачів на 52-й тиждень: {int(projected_users[51])}")
print(f"Прогнозований дохід за 6 місяців (26 тижнів): ${earned_6_months:,.2f}")
print(f"Прогнозований дохід за 12 місяців (52 тижні): ${earned_12_months:,.2f}")

--- Історичні ланцюгові коефіцієнти ---
Тиждень 1 -> 2: 0.7436
Тиждень 2 -> 3: 0.8322
Тиждень 3 -> 4: 0.8564
Тиждень 4 -> 5: 0.8887

--- Прогноз ---
Користувачів на 26-й тиждень: 146
Користувачів на 52-й тиждень: 61
Прогнозований дохід за 6 місяців (26 тижнів): $32,902.27
Прогнозований дохід за 12 місяців (52 тижні): $41,483.55
